In [1]:
import numpy as np
import csv
import os
import sys  # Import the sys module
import time
from functools import partial
from multiprocessing import Pool
from scipy.optimize import differential_evolution

sys.path.append('..')
from dataset_reader import Traces_Dataset
from DE_obj_model import de_obj_model   
from exp_hh_model import HH_model_exp

In [2]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

prestep_V_vec = dataset.prestep_V.numpy()
step_Vs_vec = dataset.step_Vs.numpy()

In [3]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit

In [ ]:
fitness = obj(params[0], *(sim_setup_2d, target_params))
print(fitness)

In [4]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [5]:
hyperparameters_grid = {
    'strategy': ['best1bin', 'best1exp', 'rand1exp', 'rand1exp', 
                'rand2bin', 'rand2exp', 'best2bin', 'best2exp',
                'randtobest1bin', 'randtobest1exp',
                'currenttobest1bin', 'currenttobest1exp'],
    'popsize': [140], #[14,28,42,56,70],  # Example popsize hyperparameter
    'mutation': [(0.1, 0.9)],  # Example mutation hyperparameter
    'recombination': [0.9],  # Example recombination hyperparameter
    'init': ['latinhypercube'],  # Example init hyperparameter
}

In [6]:
# csv_filename = "de_experiment_results_parrallell_try_1000_allmetrics.csv"
# # Define the headers for the CSV file
# csv_headers = ['Strategy', 'Popsize', 'MSE Overall Avg', 'MSE Overall Std', 'RMSE Overall Avg', 'RMSE Overall Std', 'MAE Overall Avg', 'MAE Overall Std', 'MAPE Overall Avg', 'MAPE Overall Std', 'Elapsed Time Avg',  'Elapsed Time Std']

# # Check if the CSV file exists; if not, create and write the headers
# if not os.path.exists(csv_filename):
#     with open(csv_filename, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(csv_headers)